In [1]:
import numpy as np
from math import e
import math
import random

In [65]:
class Line():
    def __init__(self, p):
        self.p1 = p[0]
        self.p2 = p[1]
        self.slope = (self.p1[1]-self.p2[1])/(self.p1[0]-self.p2[0])
        self.b = self.p1[1]-self.slope*self.p1[0]
        super().__init__()
    def calculate(self,x):
        return self.slope*x+self.b
    def plot(self):
        x = np.random.uniform(-1,1,10)
        plt.plot(x,self.calculate(x))
    def find_actual_y(self,points):
        return [np.sign(self.calculate(p[0]) - p[1]) for p in points]

    
#for layer 1, i=[0-2] and j=[0-2]
#for layer 2, i=[0-4] and j=[1]
    
class NeuralNetwork(Line):
    def __init__(self):
        self.N = 10
        self.input = 2
        self.N_var_layer1 = 3
        self.N_var_layer2 = 4
        self.N_var_layer3 = 1 #output
        self.layers = [self.N_var_layer1, self.N_var_layer2, self.N_var_layer3]
        self.n = .1
        self.N_layers = len(self.layers)
        
        #initialize weights
        self.weights1 = np.random.uniform(-1.0,1.0,(self.N_var_layer2-1,self.N_var_layer1)) # shape =(#j (outputs), #i (inputs))
        self.weights2 = np.random.uniform(-1.0,1.0,(self.N_var_layer3,self.N_var_layer2))
        self.weights = [self.weights1,self.weights2]
        
        #Create points
        self.create_line()
        X = np.random.uniform(-1.0,1.0,(self.N,self.input))
        self.y = self.find_actual_y(X)
        self.X = np.concatenate([[[1]for x in range(len(X))],X],axis=1)

        #Run SGD
        
        mse = self.mse(self.X,self.y)
        print(mse)
        
        for x in range(1):
            wt = self.vals()
            self.sgd_epoch()
            wt1 = self.vals()
            
        
        
        mse = self.mse(self.X,self.y)
        print(mse)
        
        self.eout = self.eout()
        print(self.eout)
        
    def create_line(self):
        p = np.random.uniform(-1.0,1.0,(2,2))
        while p[0][0] == p[1][0] and p[0][1] == p[1][1]:
            p = [np.random.uniform(-1.0,1.0,2) for x in range(2)]
        super().__init__(p)
         
        
        
####################################################
    def vals(self):
        change = []
        for layer in range(2,self.N_layers+1):
                layer_m1 = layer -1
                jss = [0] if layer == self.N_layers else range(self.layers[layer-1]-1)
                iss = range(self.layers[layer_m1-1]) 
                change += [self.weights[layer_m1-1][j][i] for i in iss for j in jss]
        return change 
    
    def pickRandomPoint(self):
        val = random.randint(0,self.N-1)
        return self.X[val], self.y[val]
        
    def calculate_xjLs(self,x):
        xjLs = [x]
        for layer in range(self.N_layers-1):
            x_l = [1] if layer != self.N_layers-2 else [] #self.N_layers-2 bc layer 2nd gives layer 3 and layer 2nd is actually 1
            weights_for_layer = self.weights[layer]
            for w_jl in weights_for_layer: #picking specific weights of one output (j)
                x_jl = self.tahn(np.matmul(x,w_jl.T))
                x_l.append(x_jl)
            xjLs.append(x_l)
            x = x_l
        return xjLs
    
    def tahn(self,sj):
        return (e**sj-e**-sj)/(e**sj+e**-sj)
    
    def derivative_of_tahn(self,tahn_s):
        return (1-(tahn_s)**2)
    
    def calculate_deltaL(self,x,y):
        x_1_L = self.xjLs[self.N_layers-1][0]
        return [(x_1_L-y)*self.derivative_of_tahn(x_1_L)]
    
    def calculate_deltas(self,layer,deltas_layer):
        #when layer is three gives deltas for two (already made deltaL), when layer is 2 it gives deltas for 1 --> stop at layer 2
        if layer == 2:
            return [deltas_layer]
        layer_m1 = layer-1
        d_layer = [1] if layer == self.N_layers else range(1,self.layers[layer_m1]) 
        d_layer_m1 = range(1,self.layers[layer_m1-1]) 
        
        deltas_layer_m1_i = []
        
        for i in d_layer_m1:
            sum_of_j_paths = 0
            for j in d_layer:
                deltaj = deltas_layer[j-1]
                sum_of_j_paths += deltaj*self.weights[layer_m1-1][j-1][i] 
                #j-1 because the weights for j=1 are actually the first weights as j=0 is 1
            
            delta_layer_m1_i = self.derivative_of_tahn(self.xjLs[layer_m1-1][i]) * sum_of_j_paths
            deltas_layer_m1_i.append(delta_layer_m1_i)
            
        return self.calculate_deltas(layer-1,deltas_layer_m1_i) + [deltas_layer]
        
    
####################################################
    
    
    def sgd_gradient(self):
        for layer in range(2,self.N_layers+1):
            layer_m1 = layer-1 #wij_2 where 2=layer is the weights between layer 1 (layer-1) and layer 2 (layer)
            delta_layer = self.deltas[layer-2] #delta layer 2 is the first delta at index 0
            for j in range(1,self.layers[layer-1]):
                deltaj_l = delta_layer[j-1]
                for i in range(self.layers[layer_m1-1]):
                    wij_l = self.weights[layer_m1-1][j-1][i]#weights for layer 1 to go to layer 2 is at index 0
                    xi_layer_m1 = self.xjLs[layer_m1-1][i]
                    gradient = -self.n*xi_layer_m1*deltaj_l
                    self.weights[layer_m1-1][j-1][i] = wij_l + gradient
        
        
        
    def sgd_epoch(self):
        vals = list(range(self.N))
        random.shuffle(vals)
        for v in vals:
            x, y = self.X[v],self.y[v]
            self.xjLs = self.calculate_xjLs(x)
            self.deltaL = self.calculate_deltaL(x,y)
            self.deltas = self.calculate_deltas(self.N_layers,self.deltaL)
            self.sgd_gradient()
              
    
    
######################################################
        
    def mse(self,X,y):
        w = self.weights
        return sum([(self.calculate_xjLs(X[v])[-1][0] - y[v])**2 for v in range(len(X))])/len(X)
    
    
    def eout(self):
        X = [np.random.uniform(-1.0,1.0,2) for x in range(100)]
        X = np.concatenate([[[1]for x in range(len(X))],X],axis=1)
        y = self.find_actual_y(X)
        return self.mse(X,y)



In [66]:
NeuralNetwork()

1.6452980447037924
[0.6179884974087972, -0.9111338786784737, 0.17594361393558366, -0.62277240828293, 0.18587438210634888, 0.8237992359789024, 0.31897663805073617, -0.3957047891116219, -0.8152277622567687, -0.5377258774558933, 0.33997308229687495, 0.9995786597842886, -0.500583813443986] [0.6835553557384143, -0.6829463198658073, 0.04371032036982899, -0.6508738536311175, 0.08972917694062463, 0.8951997766803389, 0.26362913029333856, -0.5671682458151135, -0.7220881332285297, -0.5377258774558933, 0.33997308229687495, 0.9995786597842886, -0.500583813443986]
1.3431672286723144
1.8231112983103543


In [230]:
sum([])

0

In [62]:
t = np.array([1,1])
y = t.copy()
print(y)
t[[0,1]] = 3
print(y)

[1 1]
[1 1]


In [281]:
i = 1
k = {"1":1}
k[str(i)]

1

In [171]:
#all that will have deltas from layer 2(the layer minus 1) is from i's 1-3 bc those are the ones where the previous layers will need their deltas

array([-0.26713872,  0.1147905 ])

In [ ]:
#j-1 because the weights for j=1 are actually the first weights from the prev layer to that layer as j=0 is 1